In [3]:
# import des bibliothèques nécessaires
import cv2

# Chemin vers le fichier vidéo  
video_path = r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\demi_finale_S2_250_Dijon_novembre_2025_start.mp4'

# Vitesse de lecture de la vidéo (1=normale, 0=pause, >1=plus rapide)
play_speed = 2


Encodage en direct des 'Points' et 'Temps mort' à partir de la lecture de la vidéo

In [4]:
# Liste pour stocker les actions
actions = []

# Dictionnaire des touches -> action
key_action_map = {
    ord('1'): 'Service J&M',
    ord('2'): 'Temps hors-jeu',
    # ord('3'): 'Service adversaire',
    # ord('7') : 'Début de set',
}

# Ouvrir la vidéo
cap = cv2.VideoCapture(video_path)

def _waitKey_fast(ms):
    # réduire le délai proportionnellement à la vitesse (au moins 1 ms)
    adj = max(1, int(ms / play_speed))
    return cv2.waitKey(adj)

if not cap.isOpened():
    print("Erreur : impossible d’ouvrir la vidéo.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_number = 0

print("\n--- LECTURE EN COURS ---\nAppuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)\n")
paused = False
last_action = ""  # Variable pour stocker la dernière action

try:
    while cap.isOpened():
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("Fin de la vidéo ou erreur de lecture.")
                break
            frame_number += 1

        # Affiche la dernière action
        if last_action and ret:
            cv2.putText(frame,
                        f"Derniere action : {last_action}",
                        (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA)

        # Indiquer mode pause
        if paused and ret:
            cv2.putText(frame,
                        "|| PAUSE ||",
                        (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 255),
                        2,
                        cv2.LINE_AA)

        if ret:
            cv2.imshow('Beach Volley Video', frame)


        key = _waitKey_fast(30) & 0xFF
        if key == ord('q'):  # touche 'q' pour quitter
            break
        elif key == ord(' '):  # barre espace
            paused = not paused
        elif key in key_action_map:
            timestamp = frame_number / fps
            timestamp_hhmmss = f"{int(timestamp // 3600):02d}:{int((timestamp % 3600) // 60):02d}:{int(timestamp % 60):02d}"
            action_name = key_action_map[key]
            last_action = action_name

            actions.append({
                # 'Time (MM:SS)': timestamp_hhmmss,
                'Frame': frame_number,
                # 'Time (s)': round(timestamp, 2),
                'Action': action_name
            })

finally:
    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()


### Ecriture des intervalles en numéro de frames dans deux fichiers textes : 1 pour les points, 1 pour les temps hors-jeu

## Fichier 'Points.txt' (format : frame_start-frame_end)
points_file = 'points_frames.txt'
# Construire les intervalles : 1 ligne = time(Point) - time(Temps hors-jeu) suivant
points_frames = []
i = 0
while i < len(actions):
    if actions[i]['Action'] == 'Point':
        start = actions[i]['Frame']
        # chercher le prochain 'Temps hors-jeu'
        j = i + 1
        end = None
        while j < len(actions):
            if actions[j]['Action'] == 'Temps hors-jeu':
                end = actions[j]['Frame']
                break
            j += 1
        if end is not None:
            points_frames.append((start, end))
            i = j + 1
        else:
            print(f"⚠️ Pas d'action 'Point' trouvée après {start:.2f}s — intervalle ignoré.")
            break
    else:
        i += 1
# Écrire les intervalles dans points_frames.txt (format : start-end))
with open(points_file, 'w', encoding='utf-8') as f:

    for start, end in points_frames:
        f.write(f"{start}-{end}\n")
print(f"✅ Fichier texte sauvegardé : {points_file}")

## Fichier 'Temps_hors_jeu.txt' (format : frame_start-frame_end)
temps_hors_jeu_file = 'temps_hors_jeu_frames.txt'
# Construire les intervalles : 1 ligne = time(Temps hors-jeu) - time(Point) suivant
temps_hors_jeu_frames = []
i = 0
while i < len(actions):
    if actions[i]['Action'] == 'Temps hors-jeu':
        start = actions[i]['Frame']
        # chercher le prochain 'Point'
        j = i + 1
        end = None
        while j < len(actions):
            if actions[j]['Action'] == 'Point':
                end = actions[j]['Frame']
                break
            j += 1
        if end is not None:
            temps_hors_jeu_frames.append((start, end))
            i = j + 1
        else:
            print(f"⚠️ Pas d'action 'Point' trouvée après {start:.2f}s — intervalle ignoré.")
            break
    else:
        i += 1
# Écrire les intervalles dans temps_hors_jeu_frames.txt (format : start-end)
with open(temps_hors_jeu_file, 'w', encoding='utf-8') as f:
    for start, end in temps_hors_jeu_frames:
        f.write(f"{start}-{end}\n")
print(f"✅ Fichier texte sauvegardé : {temps_hors_jeu_file}")



--- LECTURE EN COURS ---
Appuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)

✅ Fichier texte sauvegardé : points_frames.txt
⚠️ Pas d'action 'Point' trouvée après 1728.00s — intervalle ignoré.
✅ Fichier texte sauvegardé : temps_hors_jeu_frames.txt


Test avec encodage "Service J&M" vs "Service adv"


In [7]:
actions
points_frames

[(549, 899), (997, 1308), (1571, 1728)]

In [4]:
# Liste pour stocker les actions
actions = []

# Dictionnaire des touches -> action
key_action_map = {
    ord('1'): 'Service J&M',
    ord('2'): 'Temps hors-jeu',
    ord('3'): 'Service adversaire',
    ord('7') : 'Début de set',
}

# Ouvrir la vidéo
cap = cv2.VideoCapture(video_path)

def _waitKey_fast(ms):
    # réduire le délai proportionnellement à la vitesse (au moins 1 ms)
    adj = max(1, int(ms / play_speed))
    return cv2.waitKey(adj)

if not cap.isOpened():
    print("Erreur : impossible d’ouvrir la vidéo.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
frame_number = 0

print("\n--- LECTURE EN COURS ---\nAppuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)\n")
paused = False
last_action = ""  # Variable pour stocker la dernière action

try:
    while cap.isOpened():
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("Fin de la vidéo ou erreur de lecture.")
                break
            frame_number += 1

        # Affiche la dernière action
        if last_action and ret:
            cv2.putText(frame,
                        f"Derniere action : {last_action}",
                        (30, 40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA)

        # Indiquer mode pause
        if paused and ret:
            cv2.putText(frame,
                        "|| PAUSE ||",
                        (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 255),
                        2,
                        cv2.LINE_AA)

        if ret:
            cv2.imshow('Beach Volley Video', frame)


        key = _waitKey_fast(30) & 0xFF
        if key == ord('q'):  # touche 'q' pour quitter
            break
        elif key == ord(' '):  # barre espace
            paused = not paused
        elif key in key_action_map:
            timestamp = frame_number / fps
            timestamp_hhmmss = f"{int(timestamp // 3600):02d}:{int((timestamp % 3600) // 60):02d}:{int(timestamp % 60):02d}"
            action_name = key_action_map[key]
            last_action = action_name

            actions.append({
                # 'Time (MM:SS)': timestamp_hhmmss,
                'Frame': frame_number,
                # 'Time (s)': round(timestamp, 2),
                'Action': action_name
            })

finally:
    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()


### Création d'une liste 'points_frames' contenant les intervalles de frames correspondant aux points, avec l'information du point gagné (service précédent)


# Construire les intervalles : 1 ligne = time(Point) - time(Temps hors-jeu) suivant
points_frames = []
i = 0
while i < len(actions):
    if (actions[i]['Action'] == 'Service J&M') or (actions[i]['Action'] == 'Service adversaire'):
        start = actions[i]['Frame']
        # chercher le prochain 'Temps hors-jeu'
        j = i + 1
        end = None
        while j < len(actions):
            if actions[j]['Action'] == 'Temps hors-jeu':
                end = actions[j]['Frame']
                break
            j += 1
        if end is not None:
            points_frames.append((start, end))
            i = j + 1
        else:
            print(f"⚠️ Pas d'action 'Service' trouvée après {start:.2f}s — intervalle ignoré.")
            break
    else:
        i += 1




--- LECTURE EN COURS ---
Appuie sur une touche pour taguer une action : 1/2 (ou q pour quitter)



Découpage des segments à partir d'un fichier .txt contenant les paires de frames "Point-Temps mort"

In [5]:
# actions.insert(1, {'Frame': 200, 'Action': 'Temps hors-jeu'})
actions

[{'Frame': 179, 'Action': 'Début de set'},
 {'Frame': 521, 'Action': 'Temps hors-jeu'},
 {'Frame': 988, 'Action': 'Service adversaire'},
 {'Frame': 1204, 'Action': 'Temps hors-jeu'},
 {'Frame': 1601, 'Action': 'Service J&M'},
 {'Frame': 1839, 'Action': 'Temps hors-jeu'},
 {'Frame': 2285, 'Action': 'Service adversaire'},
 {'Frame': 2573, 'Action': 'Temps hors-jeu'},
 {'Frame': 3109, 'Action': 'Service J&M'},
 {'Frame': 3463, 'Action': 'Temps hors-jeu'},
 {'Frame': 3860, 'Action': 'Service J&M'},
 {'Frame': 4058, 'Action': 'Temps hors-jeu'}]

In [13]:
import pandas as pd

# Créer le dataframe à partir de 'actions'
list_actions = list()

i = 0
while i < len(actions):
    action = actions[i]
    
    # Si l'action n'est pas 'Temps hors-jeu', on la traite
    if action['Action'] != 'Temps hors-jeu':
        start_frame = action['Frame']
        service_side = action['Action']
        
        # Chercher le prochain 'Temps hors-jeu'
        end_frame = None
        j = i + 1
        while j < len(actions):
            if actions[j]['Action'] == 'Temps hors-jeu':
                end_frame = actions[j]['Frame']
                break
            j += 1
        
        # Ajouter à la liste si on a trouvé un 'Temps hors-jeu'
        if end_frame is not None:
            list_actions.append({
                'Service_side': service_side,
                'Start_frame': start_frame,
                'End_frame': end_frame
            })
    
    i += 1

# Convertir en DataFrame pandas
df_actions = pd.DataFrame(list_actions)

# Ajouter les colonnes de score
df_actions['Score_J&M'] = 0
df_actions['Score_adversaire'] = 0

# Mettre à jour les scores en fonction du service_side
for idx, row in df_actions.iterrows():
    if idx == 0:
        # Première ligne : initialiser selon le service
        if row['Service_side'] == 'Service J&M':
            df_actions.at[idx, 'Score_J&M'] = 1
            df_actions.at[idx, 'Score_adversaire'] = 0
        elif row['Service_side'] == 'Service adversaire':
            df_actions.at[idx, 'Score_J&M'] = 0
            df_actions.at[idx, 'Score_adversaire'] = 1
    else:
        # Lignes suivantes : reprendre le score précédent et incrémenter selon le service
        df_actions.at[idx, 'Score_J&M'] = df_actions.at[idx - 1, 'Score_J&M']
        df_actions.at[idx, 'Score_adversaire'] = df_actions.at[idx - 1, 'Score_adversaire']
        
        if row['Service_side'] == 'Service J&M':
            df_actions.at[idx, 'Score_J&M'] += 1
        elif row['Service_side'] == 'Service adversaire':
            df_actions.at[idx, 'Score_adversaire'] += 1
df_actions


# Sauvegarder le DataFrame en fichier CSV
df_actions.to_csv('actions.csv', index=False, encoding='utf-8')
print("✅ Fichier CSV sauvegardé : actions.csv")

✅ Fichier CSV sauvegardé : actions.csv


In [ ]:
import pandas as pd
df_actions=pd.read_csv('actions.csv')
df_actions

,Service_side,Start_frame,End_frame,Score_J&M,Score_adversaire
0,Début de set,179,521,0,0
1,Service adversaire,988,1204,0,1
2,Service J&M,1601,1839,1,1
3,Service adversaire,2285,2573,1,2
4,Service J&M,3109,3463,2,2
5,Service J&M,3860,4058,3,2


In [9]:
from montage_auto_frames_gpu import extract_segments_from_df_gpu


In [10]:
# def extract_segments_from_df_gpu(
#     input_video: str,
#     actions_df: pd.DataFrame,
#     output_dir: str
#     ) -> None:

#     # Construire les intervalles : 1 ligne = time(Point) - time(Temps hors-jeu) suivant
#     for _, row in actions_df.iterrows():
#         cut_point_gpu(
#             input_video=input_video,
#             start_frame=int(row["Start_frame"]),
#             end_frame=int(row["End_frame"]),
#             output_video=os.path.join(output_dir, f"extrait_{_+1:03d}.mp4")
#         )

extract_segments_from_df_gpu(
    input_video=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\demi_finale_S2_250_Dijon_novembre_2025_start.mp4',
    actions_df=df_actions,
    output_dir=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\test cut'
)

KeyError: 'Action'

In [ ]:

extract_segments_from_df_gpu(
    input_video=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\demi_finale_S2_250_Dijon_novembre_2025_start.mp4',
    actions_df=pd.read_csv('actions.csv'),
    output_dir=r'C:\Users\habib\Desktop\Montages volley et beach\Jade&Math\matchs bruts\2025 nov - S2 250 Dijon\test cut'
)

In [ ]:
import montage_auto_frames_gpu as montage_gpu

# Chemin vers le fichier vidéo
video_path = r'C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4'

## Découpages des segments 'Points'
# Paramètres
output_dir = r'C:\Users\habib\Desktop\Montages volley et beach\montage_auto\output_segments_gpu'

montage_gpu.extract_segments_gpu(video_path, points_file, output_dir=output_dir)

## Découpages des segments 'Temps hors-jeu'
# Paramètres

[INFO] Nombre de segments à extraire : 46
[GPU] Extraction frames 1427 → 1526
      Command: ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4 -vf select='between(n,1427,1526)',setpts=PTS-STARTPTS -an -c:v h264_nvenc -preset p4 -rc constqp -qp 18 C:\Users\habib\Desktop\Montages volley et beach\montage_auto\output_segments_gpu\extrait_001.mp4
[GPU] Extraction frames 1753 → 2033
      Command: ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4 -vf select='between(n,1753,2033)',setpts=PTS-STARTPTS -an -c:v h264_nvenc -preset p4 -rc constqp -qp 18 C:\Users\habib\Desktop\Montages volley et beach\montage_auto\output_segments_gpu\extrait_002.mp4
[GPU] Extraction frames 2449 → 2741
      Command: ffmpeg -y -hwaccel cuda -hwaccel_output_format cuda -i C:\Users\habib\Desktop\Montages volley et beach\montage_auto\video.mp4 -vf select='between(n,244

True